<a href="https://colab.research.google.com/github/bruno-raffa/Quantum-exercises/blob/main/Reservoir.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install dwave-ocean-sdk
! dwave setup

In [ ]:
import os
import itertools
import click
import pandas as pd
import numpy as np
from dwave.system import LeapHybridCQMSampler, LeapHybridSampler
from dimod import ConstrainedQuadraticModel, BinaryQuadraticModel, QuadraticModel, quicksum, dimod
import matplotlib.pyplot as plt

In [ ]:
# Set up scenario
num_pumps = 7
pumps = ['P'+str(p+1) for p in range(num_pumps)]
time = list(range(1, 25))
power = [15, 37, 33, 33, 22, 33, 22]
costs = [169]*7 + [283]*6 + [169]*3 + [336]*5 + [169]*3
flow = [75, 133, 157, 176, 59, 69, 120]
demand = [44.62, 31.27, 26.22, 27.51, 31.50, 46.18, 69.47, 100.36, 131.85, 
            148.51, 149.89, 142.21, 132.09, 129.29, 124.06, 114.68, 109.33, 
            115.76, 126.95, 131.48, 138.86, 131.91, 111.53, 70.43]
            
v_init = 550
v_min = 523.5
v_max = 1500

In [ ]:
def build_cqm(num_pumps=num_pumps, costs=costs, flow=flow, demand=demand):

  print("\nBuilding a CQM for {} pumps.".format(str(num_pumps)))
  cqm = ConstrainedQuadraticModel()

  # Objective is to minimize the total costs
  objective = quicksum(x[p][t] *  power[p] * costs[t]/1000 for t in range(len(time)) for p in range(num_pumps))
  cqm.set_objective(objective)

  #Constraint 1: Every pump runs at least once per day
  for p in range(num_pumps):
    cqm.add_constraint(
      quicksum(x[p][t] for t in range(len(time))) >= 1
      )

  #Constraint 2: At most num_pumps-1 pumps per time slot (1 pump kept off as a backup)
  for t in range(len(time)):
    cqm.add_constraint(
      quicksum(x[p][t] for p in range(num_pumps)) <= (num_pumps - 1)
      )
    
  #Constraint 3: Satisfy a reosurce demand varying with time and level maintained within a specific range
  for t in range(len(time)):
    water_level = v_init - sum(demand[0:t+1])
    cqm.add_constraint(
    quicksum(x[p][k]*(flow[p]) for p in range(num_pumps) for k in range(t+1)) >= v_min - water_level
    )

  for t in range(len(time)):
      water_level = v_init - sum(demand[0:t+1])
      cqm.add_constraint(
      quicksum(x[p][k]*(flow[p]) for p in range(num_pumps) for k in range(t+1)) <= v_max - water_level
      )

  return cqm

In [ ]:
def parse_solution(sampleset, costs=costs, flow=flow):

  feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

  if not len(feasible_sampleset):
      raise ValueError("No feasible solution found")

  print("\nProcessing sampleset returned...")

  y = [['P' + str(p) + '_' + str(t) for t in time] for p in range(num_pumps)]
  best = feasible_sampleset.first


  verbose = True
  total_flow = 0
  total_cost = 0
  # Print out time slots header
  if verbose:
      timeslots = "\n\t" + "\t".join(str(t) for t in time)
      print(timeslots)
  for p in range(num_pumps):
    printout = str(pumps[p])
    for t in range(len(time)):
        printout += "\t" + str(best.sample[y[p][t]])
        total_flow += best.sample[y[p][t]] * flow[p]
        total_cost += best.sample[y[p][t]] * costs[t] * power[p] / 1000
    if verbose:
        print(printout)


  # Generate printout for general water levels
  printout = "Level:\t"
  reservoir = [v_init]
  pump_flow_schedule = []
  for t in range(len(time)):
      hourly_flow = reservoir[-1]
      for p in range(num_pumps):
          hourly_flow += best.sample[y[p][t]] * flow[p]
      reservoir.append(hourly_flow-demand[t])
      pump_flow_schedule.append(hourly_flow - reservoir[-2])
      printout += str(int(reservoir[-1])) + "\t"
  if verbose:
      print("\n" + printout)

  # Print out total flow and cost information
  print("\nTotal flow:\t", total_flow)
  print("Total cost:\t", total_cost, "\n")

    

In [ ]:
# Build a variable for each pump
x = [[dimod.Binary(f'P{p}_{t}') for t in time] for p in range(num_pumps)]

# Initialize CQM
cqm = build_cqm()

# Instantiate the sampler
sampler = LeapHybridCQMSampler()
print("Submitting CQM to solver {}.".format(sampler.solver.name))
sampleset = sampler.sample_cqm(cqm, label='Example - Reservoir')


Building a CQM for 7 pumps.
Submitting CQM to solver hybrid_constrained_quadratic_model_version1.


In [ ]:
parse_solution(sampleset)


Processing sampleset returned...

	1	2	3	4	5	6	7	8	9	10	11	12	13	14	15	16	17	18	19	20	21	22	23	24
P1	0.0	1.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0
P2	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
P3	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
P4	0.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	1.0	1.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0
P5	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0
P6	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0
P7	1.0	1.0	1.0	0.0	1.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	1.0	1.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0	0.0

Level:	625	789	882	988	1076	1225	1332	1231	1100	951	801	659	527	920	1091	1153	1043	928	801	669	530	650	597	527	

Total flow:	 2363.0
Total cost:	 82.64100000000002 

